##This notebook will import multiple FED3 files and concatenate them by the following rules:
- The columns "Left_Count", "Right_Count", and "Pellet_Count" will increment across files as if they were collected in one session
- Files will be ordered chronologically by their dates in the column "MM:DD:YY hh:mm:ss"
- Trying to concatenate files with different "Session_type" values will result in an error
- Trying to concatenate files that are not contiguous in time by >24h will result in a warning

Written by Gargi Chatterjee Basu/Lex Kravitz with ChatGPT 4o, Jan 2025

A single .py script to run this code offline can be found [here](
https://github.com/KravitzLab/KreedLabWiki/blob/main/PythonCode/ConcatenateFEDfiles.py) and at the last cell in this notebook

## Step 1: Upload FED3 files to concatenate

In [5]:
import pandas as pd
import io
from google.colab import files

# Upload CSV files
uploaded = files.upload()

# Store uploaded file names
file_paths = list(uploaded.keys())



Saving FED003_010725_03.CSV to FED003_010725_03.CSV
Saving FED002_010725_concat.csv to FED002_010725_concat.csv


## Step 2: Concatenate files

In [7]:
def process_files(file_paths):
    df_combined = pd.DataFrame()
    file_info = []

    # Loop through the files to get the first timestamp of each file
    for file_path in file_paths:
        df = pd.read_csv(io.StringIO(uploaded[file_path].decode('utf-8')))
        df['MM:DD:YYYY hh:mm:ss'] = pd.to_datetime(df['MM:DD:YYYY hh:mm:ss'])
        first_timestamp = df['MM:DD:YYYY hh:mm:ss'].iloc[0]
        file_info.append((first_timestamp, file_path))

    # Sort files by first timestamp
    file_info.sort(key=lambda x: x[0])
    file_paths_sorted = [file_path for _, file_path in file_info]

    # Read first file and initialize tracking
    df_combined = pd.read_csv(io.StringIO(uploaded[file_paths_sorted[0]].decode('utf-8')))
    df_combined['MM:DD:YYYY hh:mm:ss'] = pd.to_datetime(df_combined['MM:DD:YYYY hh:mm:ss'])
    left_poke_max = df_combined['Left_Poke_Count'].max()
    right_poke_max = df_combined['Right_Poke_Count'].max()
    pellet_count_max = df_combined['Pellet_Count'].max()

    for file_path in file_paths_sorted[1:]:
        df_new = pd.read_csv(io.StringIO(uploaded[file_path].decode('utf-8')))
        df_new['MM:DD:YYYY hh:mm:ss'] = pd.to_datetime(df_new['MM:DD:YYYY hh:mm:ss'])

        # Ensure session types match
        if df_combined['Session_type'].iloc[0] != df_new['Session_type'].iloc[0]:
            raise ValueError(f"Error: Session types are different between files! Cannot concatenate {file_path}.")

        # Check timestamp continuity
        time_gap = df_new['MM:DD:YYYY hh:mm:ss'].iloc[0] - df_combined['MM:DD:YYYY hh:mm:ss'].iloc[-1]
        if time_gap > pd.Timedelta(hours=24):
            print(f"Warning: Timestamps are not contiguous between files, please check {file_path}")

        # Adjust counts
        df_new['Left_Poke_Count'] += left_poke_max
        df_new['Right_Poke_Count'] += right_poke_max
        df_new['Pellet_Count'] += pellet_count_max

        # Concatenate data
        df_combined = pd.concat([df_combined, df_new], ignore_index=True)

        # Update tracking values
        left_poke_max = df_combined['Left_Poke_Count'].max()
        right_poke_max = df_combined['Right_Poke_Count'].max()
        pellet_count_max = df_combined['Pellet_Count'].max()

    return df_combined

# Process files
print(f"Succesfiully concatened files!")
df_combined = process_files(file_paths)

Succesfiully concatened files!


## Step 3: Download concatenated FED3 file

In [8]:
# Save and provide download link
output_file_name = "concatenated_data.csv"
df_combined.to_csv(output_file_name, index=False)
files.download(output_file_name)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Single .py script is below for running offline

In [ ]:
import pandas as pd

# List of CSV files to concatenate
file_paths = [
    r'\FED_001.csv',
    r'\FED_002.csv'
    # Add more files here as needed
]

#what to call the output file
output_file = r'concat.csv'

# Initialize an empty DataFrame for concatenation
df_combined = pd.DataFrame()

# List to hold (timestamp, file_path) tuples
file_info = []

# Loop through the files to get the first timestamp of each file
for file_path in file_paths:
    df = pd.read_csv(file_path, header=0)  # Read the file
    df['MM:DD:YYYY hh:mm:ss'] = pd.to_datetime(df['MM:DD:YYYY hh:mm:ss'])  # Convert timestamp to datetime
    first_timestamp = df['MM:DD:YYYY hh:mm:ss'].iloc[0]  # Get the first timestamp
    file_info.append((first_timestamp, file_path))  # Store the timestamp and file path

# Sort the files by the first timestamp (chronologically)
file_info.sort(key=lambda x: x[0])

# Reorder the file_paths based on the sorted timestamps
file_paths_sorted = [file_path for _, file_path in file_info]

# Initialize the combined DataFrame with the first sorted file
df_combined = pd.read_csv(file_paths_sorted[0], header=0)
df_combined['MM:DD:YYYY hh:mm:ss'] = pd.to_datetime(df_combined['MM:DD:YYYY hh:mm:ss'])

# Track the current maximum values of the counting columns
left_poke_max = df_combined['Left_Poke_Count'].max()
right_poke_max = df_combined['Right_Poke_Count'].max()
pellet_count_max = df_combined['Pellet_Count'].max()

# Loop through the rest of the sorted files
for file_path in file_paths_sorted[1:]:
    # Read the next file without headers
    df_new = pd.read_csv(file_path, header=0)

    # Ensure 'timestamp' column is in datetime format for the new file
    df_new['MM:DD:YYYY hh:mm:ss'] = pd.to_datetime(df_new['MM:DD:YYYY hh:mm:ss'])

    # Check if session types are the same in both files
    if df_combined['Session_type'].iloc[0] != df_new['Session_type'].iloc[0]:
        raise ValueError(f"Error: Session types are different between files! Cannot concatenate {file_path}.")

    # Check if the time gap between the last timestamp in the combined DataFrame and the first timestamp in the new file is greater than 24 hours
    time_gap = df_new['MM:DD:YYYY hh:mm:ss'].iloc[0] - df_combined['MM:DD:YYYY hh:mm:ss'].iloc[-1]
    if time_gap > pd.Timedelta(hours=24):
        print(f"Warning: Timestamps are not contiguous between {file_paths[0]} and {file_path}, please check the files")

    # Increment the count for the new DataFrame
    df_new['Left_Poke_Count'] += left_poke_max
    df_new['Right_Poke_Count'] += right_poke_max
    df_new['Pellet_Count'] += pellet_count_max

    # Append the new DataFrame to the combined DataFrame
    df_combined = pd.concat([df_combined, df_new], ignore_index=True)

    # Update the max values for the next file
    left_poke_max = df_combined['Left_Poke_Count'].max()
    right_poke_max = df_combined['Right_Poke_Count'].max()
    pellet_count_max = df_combined['Pellet_Count'].max()

# Save the concatenated DataFrame to a new CSV file in the desired folder
df_combined.to_csv(output_file, index=False)

print("CSV files have been concatenated successfully!")
